# Training a Variational Autoencoder for jet images with  **Conv 2D**  

---
In this notebook, we train a Variational Autoencoder on a dataset of QCD jet images.
Both the encoder and decoder networks are based on Conv2D architectures. 
This autoencoder can be used
- as a clustering algorithm in the latent space
- as a compression algorithm
- as a generator 
- as an anomaly detection algorithm

Some of these functionalities are exploited below.

The notebook starts prpearing the QCD dataset and a set of
"anomalous jets" (boosted W-, Z-, and top-jets).
The VAE is trained on a training+validation dataset. Then, 
it is used on the test dataset for some of the applications listed above. Due to
the low statistics available, the validation dataset is recycled
as a test dataset. 

For details on the dataset, see https://github.com/pierinim/tutorials/blob/master/HiggsSchool/Lecture1/Notebook1_ExploreDataset.ipynb

---

In [ ]:
import os
import h5py
import glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

# Preparation of the training and validation samples

---
In order to import the dataset, we now
- clone the dataset repository (to import the data in Colab)
- load the h5 files in the data/ repository
- extract the data we need: a target and jetImage 

To type shell commands, we start the command line with !

In [ ]:
! git clone https://github.com/pierinim/tutorials.git

In [ ]:
! ls tutorials/Data/JetDataset/

In [ ]:
import glob
target = np.array([])
jetImage = np.array([])
#----------
# if you are running locally, you should do
# datafiles = glob.glob("../Data/JetDataset/*h5") 
#----------
# on Colab
#----------
datafiles = glob.glob("tutorials/Data/JetDataset/*h5") 
for fileIN in datafiles:
    print("Appending %s" %fileIN)
    f = h5py.File(fileIN, "r")
    myjetImage = np.array(f.get("jetImage"), np.float32)
    mytarget = np.array(f.get('jets')[0:,-6:-1], np.float32)
    jetImage = np.concatenate([jetImage, myjetImage], axis=0) if jetImage.size else myjetImage
    target = np.concatenate([target, mytarget], axis=0) if target.size else mytarget
    f.close()
print(target.shape, jetImage.shape)

In keras, images are representable as $n \times m \times k$ tensors, where $n \times m$ are the pixel dimenions and $k$ is the number of channels (e.g., 1 in a black\&while image, 3 for an RGB image). In our case, k=1. To comply to this, we add the channel index by reshaping the image dataset

In [ ]:
jetImage = jetImage.reshape((jetImage.shape[0], jetImage.shape[1], jetImage.shape[2], 1))
print(jetImage.shape)

We now separate the dataset in 4:
- a training dataset, consisting of quarks and gluons
- three 'anomalous jets' samples: W, Z, and top

In [ ]:
jetImage_standard = jetImage[np.argmax(target,axis=1)<2]
jetImage_W = jetImage[np.argmax(target,axis=1)==2]
jetImage_Z = jetImage[np.argmax(target,axis=1)==3]
jetImage_t = jetImage[np.argmax(target,axis=1)==4]
print(jetImage_standard.shape, jetImage_W.shape, jetImage_Z.shape, jetImage_t.shape)

Notice that this is an unsupervised algorithm, so we don't need the target array anymore.
Nevertheless, we keep a part of it around, since it might be useful to test the response 
of the algorithm to quarks and gluons separetly

In [ ]:
label_standard = target[np.argmax(target,axis=1)<2]
# some cleanup to save memory
del target
del jetImage

We now shuffle the standard-jet data and its labels, splitting them into a training, a validation+test dataset with 2:1:1 ratio. 

Then we separate the validation+test in two halves (training and validation)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, label_train, label_val = t= train_test_split(jetImage_standard, label_standard, test_size=0.5)
print(X_train.shape, X_val.shape, label_train.shape, label_val.shape)
len_val = X_val.shape[0]
X_test = X_val[int(len_val/2.):,:,:,:]
label_test = label_val[int(len_val/2.):,:]
X_val = X_val[:int(len_val/2.),:,:,:]
label_test = label_val[:int(len_val/2.),:]
print(X_train.shape, X_val.shape, X_test.shape, label_train.shape, label_val.shape, label_test.shape)

In [ ]:
# some more cleanup
del jetImage_standard

# Building the ConVAE model

In [ ]:
# keras imports
from keras.models import Model
from keras.layers import Dense, Input, Conv2D, Flatten, Activation, Deconv2D, Lambda
from keras.layers import MaxPooling2D, BatchNormalization, Activation, Reshape, UpSampling2D
from keras.utils import plot_model
from keras import backend as K
from keras import metrics
from keras.losses import mse
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
import tensorflow as tf

In [ ]:
img_rows = X_train.shape[1]
img_cols = X_train.shape[2]
image_shape = (img_rows, img_cols, 1)
latent_dim = 5

In [ ]:
#---------------------------
# Normal Sampling function for latent space
#---------------------------
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


#---------
# Enncoder
#---------
inputImage = Input(shape=(image_shape))
#
x = Conv2D(10, kernel_size=(5, 5), data_format="channels_last", strides=(1, 1), padding="same")(inputImage)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D( pool_size = (5, 5))(x)
#
x = Conv2D(15, kernel_size=(4, 4), data_format="channels_last", strides=(1, 1), padding="same")(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D( pool_size = (5, 5))(x)
#
x = Conv2D(20, kernel_size=(4, 4), data_format="channels_last", strides=(1, 1), padding="same")(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D( pool_size = (4, 4))(x)
#
x = Flatten()(x)
x = Dense(10, activation='relu')(x)
z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
encoder = Model(inputImage, [z_mean, z_log_var, z], name='encoder')
print("=== ENCODER ===")
encoder.summary()

#---------
# Decoder
#---------
latent_input = Input(shape=(latent_dim,))
x = Dense(20, activation='relu')(latent_input)
x = Reshape((1, 1, 20))(x)
#
x = UpSampling2D((4, 4))(x)
x = BatchNormalization()(x)
x = Deconv2D(15, kernel_size=(4,4), data_format="channels_last", strides=(1, 1), padding="same")(x)
x = Activation('relu')(x)
#
x = UpSampling2D((5, 5))(x)
x = BatchNormalization()(x)
x = Deconv2D(10, kernel_size=(4,4), data_format="channels_last", strides=(1, 1), padding="same")(x)
x = Activation('relu')(x)
#
x = UpSampling2D((5, 5))(x)
x = BatchNormalization()(x)
x = Deconv2D(1, kernel_size=(4,4), data_format="channels_last", strides=(1, 1), padding="same")(x)
dec = Activation('relu')(x)
decoder = Model(latent_input, dec, name = 'decoder')
print("=== DECODER ===")
decoder.summary()

# instantiate VAE model
outputs = decoder(encoder(inputImage)[2])
vae = Model(inputImage, outputs, name='vae')

In [ ]:
def mean_squared_error(y_true, y_pred):
    return mse(tf.reshape(y_true, [-1,100*100]), tf.reshape(y_pred, [-1, 100*100]))

def vae_loss(y_true, y_pred):
    reconstruction_loss = mean_squared_error(y_true, y_pred)
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    return K.mean(reconstruction_loss - 0.5 * kl_loss)

vae.compile(optimizer='adam', loss=vae_loss)
vae.summary()

We now train the model. Notice the difference with respect to the supervised case
- the input to the training is (X,X) and nor (X, y). Similarly for the validation dataset
- the model has no dropout. It is difficult for an unsupervised model to overtran, so there is not really a need

In [ ]:
batch_size = 128
n_epochs = 100

In [ ]:
# train 
history = vae.fit(X_train, X_train, epochs=n_epochs, batch_size=batch_size, verbose = 2,
                validation_data=(X_val, X_val),
                callbacks = [
                EarlyStopping(monitor='val_loss', patience=10, verbose=1),
                ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1),
                TerminateOnNaN()])

In [ ]:
# plot training history
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.yscale('log')
plt.title('Training History')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='upper right')
plt.show()

### NOTE: 
To save the model, you can do (not needed here):

`model_json = autoencoder.to_json()
with open("tutorials/HiggsSchool/models/jetAE_Conv2D.json", "w") as json_file:
    json_file.write(model_json)
autoencoder.save_weights("tutorials/HiggsSchool/models/jetAE_Conv2D.h5")`

# Anomaly Detection

In order to detect anomalous jets, we build the distribution of the reconstruction loss (the MSE term) and define as anomaly all the jets falling in the tail, at some fixed p-value. 
To evaluate the strength of the test, we can consider a set of anomalies and evaluate the ROC curves

### Loss Distributions

In [ ]:
labels = ['W', 'Z', 'top']

In [ ]:
anomaly = [jetImage_W, jetImage_Z, jetImage_t]
predictedQCD = vae.predict(X_test)
predicted_anomaly = []
for i in range(len(labels)):
    predicted_anomaly.append(vae.predict(anomaly[i]))

In [ ]:
def mse(image_in, image_out):
    mse = (image_out-image_in)*(image_out-image_in)
    # sum over channel
    mse = np.sum(mse,axis=-1)
    # sum over y
    mse = np.sum(mse, axis=-1)
    # sum over x
    mse = np.sum(mse, axis=-1)
    return mse    

In [ ]:
lossQCD = mse(X_test, predictedQCD)
loss_anomaly = []
for i in range(len(labels)):
    loss_anomaly.append(mse(anomaly[i], predicted_anomaly[i]))

In [ ]:
maxScore = np.max(lossQCD)
# plot QCD
plt.figure()
plt.hist(lossQCD, bins=100, label='QCD', density=True, range=(0, maxScore), 
         histtype='step', fill=False, linewidth=1.5)
for i in range(len(labels)):
    plt.hist(loss_anomaly[i], bins=100, label=labels[i], density=True, range=(0, maxScore),
            histtype='step', fill=False, linewidth=1.5)
plt.semilogy()
plt.xlabel("VAE Loss")
plt.ylabel("Probability (a.u.)")
plt.grid(True)
plt.legend(loc='upper right')
plt.show()

### Building the ROC Curves

In [ ]:
from sklearn.metrics import roc_curve, auc
plt.figure()
targetQCD = np.zeros(lossQCD.shape[0])
for i, label in enumerate(labels):
        print(loss_anomaly[i].shape, targetQCD.shape)
        trueVal = np.concatenate((np.ones(loss_anomaly[i].shape[0]),targetQCD))
        predVal = np.concatenate((loss_anomaly[i],lossQCD))
        print(trueVal.shape, predVal.shape)
        fpr, tpr, threshold = roc_curve(trueVal,predVal)
        auc1= auc(fpr, tpr)
        plt.plot(tpr,fpr,label='%s Anomaly Detection, auc = %.1f%%'%(label,auc1*100.))
plt.semilogy()
plt.xlabel("sig. efficiency")
plt.ylabel("bkg. mistag rate")
plt.grid(True)
plt.legend(loc='lower right')
plt.show()

# Generator

Sampling randome numbers from Gaus(0,1), we can generate a sample
of points in the latent space, which we can pass to the decoder to
generate jet images

In [ ]:
# generate a sample of Gaussian numbers, with mean = 0 and sigma = 1
z_gen = np.random.normal(0., 1., (X_val.shape[0],latent_dim))

In [ ]:
#apply the decoder on these points of the latent space
X_gen = decoder.predict(z_gen)
print(X_gen.shape)

In [ ]:
# Average Image
from matplotlib.colors import LogNorm
labelCat= ["gluon", "quark", "W", "Z", "top"]
images = [X_val[:,:,:,0], X_gen[:,:,:,0]]
labelAveImage = ["Input","VAE"]
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
for i in range(len(images)):
    SUM_Image = np.sum(images[i], axis = 0)
    plt.imshow(SUM_Image/float(images[i].shape[0]), origin='lower',norm=LogNorm(vmin=0.01))
    plt.colorbar()
    plt.title(labelAveImage[i], fontsize=15)
    plt.xlabel("$\Delta\eta$ cell", fontsize=15)
    plt.ylabel("$\Delta\phi$ cell", fontsize=15)
    plt.show()

In [ ]:
from scipy.stats import moment
# compute some quantity that we want to plot
def moments(myImages, projAxis):
    xProj = np.sum(myImages, axis=projAxis)
    moments = moment(xProj, moment = 1)
    rms = np.sqrt(moment(xProj, moment = 2))
    moments = np.concatenate((moments, rms), axis = 1)
    moments = np.concatenate((moments, moment(xProj, moment = 3)), axis = 1)
    moments = np.concatenate((moments, moment(xProj, moment = 4)), axis = 1)
    return moments 

In [ ]:
# plot X and Y moments of the generated vs original distributions
p_label = ["E[$\eta$]", "E[$\eta^2$]", "E[$\eta^3$]", "E[$\eta^4$]",
          "E[$\phi$]", "E[$\phi^2$]", "E[$\phi^3$]", "E[$\phi^4$]",]
p_gen = moments(X_gen, 1)
p_gen = np.concatenate((p_gen, moments(X_gen, 2)), axis = 1)
p_val = moments(X_val, 1)
p_val = np.concatenate((p_val, moments(X_val, 2)), axis = 1)

for i in range(p_gen.shape[1]):
    f = plt.figure(i)
    xmin = np.min(p_val[:,i])
    xmax = np.max(p_val[:,i])    
    plt.hist(p_val[:,i], bins=30, range=(xmin,xmax), histtype='step', fill=False, linewidth=1.5, density=True, label="Input")
    plt.hist(p_gen[:,i], bins=30, range=(xmin,xmax), histtype='step', fill=False, linewidth=1.5, density=True, label="VAE")
    plt.xlabel(p_label[i])
    plt.yscale('log')
    plt.legend(loc='upper right')
    plt.show()